In [2]:
import sys
import os
import numpy as np 
from scipy.integrate import solve_ivp
from sklearn.linear_model import BayesianRidge


sys.path.append(os.path.abspath('../packages'))
from utilities import *

In [3]:
path_dir_script = '../packages'

## EGFR NETWORK with Michaelis-Menten Dynamics without enzyme units
EGFR_M_S_UNITS_PATH =path_dir_script + '/parameters/EGFR/egfr_m_s_units.csv'

## Defining the paths for the EGFR network with Michaelis-Menten Dynamics
M_DYNB_PATH =path_dir_script + '/parameters/EGFR/bdyn.txt'
M_DYNH_PATH= path_dir_script + '/parameters/EGFR/hdyn.txt'
M_DYNU_PATH= path_dir_script + '/parameters/EGFR/udyn.txt'
M_DYNM_PATH= path_dir_script + '/parameters/EGFR/mdyn.txt'



In [4]:
# Calculates the dynamics of the system

dyn, S, flux = get_dynamics(network='', dynb_path=M_DYNB_PATH, dynh_path=M_DYNH_PATH, dynu_path=M_DYNU_PATH, dynm_path=M_DYNM_PATH, units_path= EGFR_M_S_UNITS_PATH)

# Loading steady state values for units 
y_val = load_rubin14_MM_yvals(y_vals_file='../packages/parameters/EGFR/yvals.txt')

In [5]:
# Setting dynamics of the system and updating the parameters

par = make_par('egfr')
par['dynamics'] = dyn
par = refresh_netw_par(par)

In [ ]:
# Create initial conditions for concentrations of units, perturbed from y_val
stdv = .2   # standard deviation from y_val
init = y_val * ( 1 +  np.random.normal(0, stdv, len(y_val)))    # initial conditions

In [7]:
n_runs = 20
t_eval = np.linspace(0, 20, 100)

# Generating Data 
init_list = [y_val * ( 1 +  np.random.normal(0, stdv, len(y_val))) for i in range(n_runs)]

d1 = None 
der = None 

for r in range(n_runs): 
    sol = solve_ivp(dyn,t_span = (0, t_eval[-1]),  t_eval=t_eval, y0= init_list[r], method='BDF')  # solve the system of ODEs
    x = sol.y[par['idx_subnw'], :]  # extract the concentrations of the observed units 

    if (d1 is not None) and (der is not None): 
        d1 = np.append(d1, design_matrix(x.T), axis=0)
        der = np.append(der, np.gradient(x,t_eval,  axis=1), axis=1)
    else:     
        d1 = design_matrix(x.T)
        der = np.gradient(x,t_eval,  axis=1)    


# Fitting the model
alphas = None 
betas = None 
weights = None
Sigmas = None

for i in range(len(par['units_subnw'])): 
    clf = BayesianRidge(lambda_1=0, lambda_2=0, alpha_1=0, alpha_2=0, copy_X = True, tol=1e-4, fit_intercept=False)
    clf.fit(d1, der[i, :])

    if (alphas is not None) and (betas is not None) and (weights is not None): 
        alphas = np.append(alphas, [clf.lambda_], axis=0)
        betas = np.append(betas, [clf.alpha_], axis=0)
        weights = np.append(weights, [clf.coef_], axis=0)
#        Sigmas = np.append(Sigmas, clf.sigma_, axis=0)

    else: 
        alphas = [clf.lambda_]
        betas = [clf.alpha_]
        weights = [clf.coef_]

In [8]:
def get_stoch_term(epsilon, S, flux):
    g = lambda t, stat: np.sqrt(epsilon) * S @ np.diag(np.sqrt(flux(t, stat))) 
    
    return g


det_dyn, S, flux = get_dynamics()
stoch_dyn = get_stoch_term(0.00001, S, flux)


In [9]:
def int_ito(f, g, y0, t_eval, max_reject=100, n_lower_dt=10):
    
    n_steps = len(t_eval)
    N_w = g(0, y0).shape[1]


    y = np.zeros((len(y0), n_steps))
    y[:, 0] = y0
    dt = t_eval[1] - t_eval[0]
    for i in range(1, n_steps):
        val = -1

        counter = 0

        while np.any(val< 0): 
            if counter > max_reject : 
                raise RuntimeError('Max reject reached')                
            elif counter > n_lower_dt: 
                print('lower dt')    
                _t_eval = np.linspace(t_eval[i-1], t_eval[i], n_lower_dt)
                val = int_ito(f, g, y[:, i-1], _t_eval, max_reject=n_lower_dt)[:, -1]
            else:
                val = y[:, i-1] + f(t_eval[i-1], y[:, i-1]) * dt + g(t_eval[i-1], y[:, i-1]) @ np.random.normal(0, 1, N_w) * np.sqrt(dt)

            y[:, i] = val
            counter += 1

    return y

In [10]:
sol = int_ito(det_dyn, stoch_dyn, init_list[0], np.linspace(0, 20, 100))

In [11]:
import tqdm 

In [37]:
from tqdm import tqdm
n_epsilon = 3
epsilon_list = np.logspace(-8, -2, base=8, num=n_epsilon)
dt_list = [5e-3, 5e-2]
n_runs = 2
n_avg = 3
T = 10
stdv = .2

n_dt = len(dt_list)
t_eval = [np.linspace(0, T, int(np.ceil(T / dt))) for dt in dt_list]

n_sub = len(par['units_subnw'])

# Generating Data 
init_list = [[y_val * ( 1 +  np.random.normal(0, stdv, len(y_val))) for i in range(n_runs)] for _ in range(n_avg)]



alpha_list = np.empty((n_epsilon, n_dt, n_avg, n_sub ))
beta_list  = np.empty((n_epsilon, n_dt, n_avg, n_sub ))
weights_list = np.empty((n_epsilon, n_dt, n_avg, n_sub ), dtype=object)
alpha_error_list = np.empty((n_epsilon, n_dt, n_avg, n_sub ))
beta_error_list = np.empty((n_epsilon, n_dt, n_avg, n_sub ))
emp_var_list = np.empty((n_epsilon, n_dt, n_avg, n_sub))


bar_e = tqdm(enumerate(epsilon_list), total=len(epsilon_list))
for i_e, epsilon in bar_e:
    bar_e.set_description(f'epsilon: {epsilon:.1e}')    
    det_dyn, S, flux = get_dynamics()
    stoch_dyn = get_stoch_term(epsilon, S, flux)
     

    for i_a in range(n_avg):        
        
        d1 = [None]*n_dt
        der = [None]*n_dt
        der_det = [None]*n_dt



        bar_runs = tqdm(range(n_runs), total=n_runs, leave=False)
        for r in bar_runs:
            init = init_list[i_a][r]

            sol_det = solve_ivp(dyn,t_span = (0, T),  t_eval=t_eval[0], y0= init, method='BDF')  # solve the system of ODEs
            sol_stoch = int_ito(det_dyn, stoch_dyn, init, t_eval=t_eval[0])

            x_det = sol_det.y[par['idx_subnw'], :]  # extract the concentrations of the observed units 
            x_stoch = sol_stoch[par['idx_subnw'], :]  # extract the concentrations of the observed units 
            
           
            for i_dt, _ in enumerate(dt_list): 
                x_s = x_stoch[:, ::int(np.ceil(dt_list[i_dt] / dt_list[0]))]
                x_d = x_det[:, ::int(np.ceil(dt_list[i_dt] / dt_list[0]))]

                if (d1[i_dt] is not None): 
                    d1[i_dt] = np.append(d1[i_dt], design_matrix(x_s.T), axis=0)
                    der[i_dt] = np.append(der[i_dt], np.gradient(x_s,t_eval[i_dt],  axis=1), axis=1)                                        
                    der_det[i_dt] = np.append(der_det[i_dt], np.gradient(x_d, t_eval[i_dt],  axis=1), axis=1)                    
                                        
                                    
                else:     
                    d1[i_dt] = design_matrix(x_s.T)
                    der[i_dt] = np.gradient(x_s,t_eval[i_dt],  axis=1)    
                    der_det[i_dt] = np.gradient(x_d,t_eval[i_dt],  axis=1)  
                
        for i_dt in range(n_dt):
               
            emp_var_list[i_e, i_dt, i_a] = np.var(der[i_dt] - der_det[i_dt], axis=1) 

        
        for i_dt in range(n_dt):
            for i in range(len(par['units_subnw'])): 
                clf = BayesianRidge(lambda_1=0, lambda_2=0, alpha_1=0, alpha_2=0, copy_X = True, tol=1e-4, fit_intercept=False)
                clf.fit(d1[i_dt], der[i_dt][i])

                alpha = clf.lambda_
                beta = clf.alpha_            

                temp = second_der_alpha_beta_2(alpha, beta, d1[i_dt], der[i_dt][i])

                alpha_error_list[i_e, i_dt, i_a, i] = np.abs(temp[1]/(temp[0]*temp[1]-temp[2]**2))**0.5
                beta_error_list[i_e,i_dt, i_a, i]  = np.abs(temp[0]/(temp[0]*temp[1]-temp[2]**2))**0.5                
                
                alpha_list[i_e, i_dt,  i_a, i] = clf.lambda_
                beta_list[i_e,i_dt,  i_a, i] = clf.alpha_
                weights_list[i_e,i_dt, i_a, i] = clf.coef_

                            
                




            

epsilon: 6.0e-08:   0%|          | 0/3 [00:00<?, ?it/s]

epsilon: 1.6e-02: 100%|██████████| 3/3 [00:20<00:00,  6.79s/it]


In [38]:
t1_data = {'alpha_list': alpha_list, 'beta_list': beta_list, 'weights_list': weights_list, 'alpha_error_list': alpha_error_list, 'beta_error_list': beta_error_list, 'emp_var_list': emp_var_list, 'epsilon_list': epsilon_list, 'dt_list': dt_list, 'n_runs': n_runs, 'n_avg': n_avg, 'T': T, 'stdv': stdv}

np.save('../data/09_EGFR_MM_stoch', t1_data)

In [14]:
def second_der_alpha_beta_2(alpha, beta, d1, x):
    
    """
    Returns the second derivative of the log-likelihood per number of data points, of the training data coming from a Gaussian process with weight correlation alpha and random noise beta
    
    Input:
    par       = par[0]: alpha value, par[1]: beta value
    d1        = design matrix
    num_subnw = num of species in the subnetwork
    obs       = the observations, that is the time derivatives of the concentrations at all time points.
    method    = if log, then it return the gradient wrt log alpha and log beta
    
    Output:
    
    The second derivative of log_likelihood per number of training points at given alpha and beta. dim = 3: wrt alpha, beta and mixed term.
    
  
    """

    
    num_basis = d1.shape[1] #int(num_subnw*(num_subnw+3)/2 + 1)
    num_obs   = len(x)
    A_inv     = np.linalg.inv(np.diag(alpha*np.ones(num_basis)) + beta*np.matmul(d1.T,d1))
    mn        = beta*np.matmul(A_inv,np.matmul(d1.T,x))
    
    g2_alpha      = -0.5*num_basis/alpha**2 + np.matmul(mn.T,np.matmul(A_inv,mn)) + 0.5*np.trace(np.linalg.matrix_power(A_inv,2))
    g2_beta       = -0.5*num_obs/beta**2 - np.matmul((x-np.matmul(d1,mn)).T,(np.matmul(d1,np.matmul(A_inv,np.matmul(d1.T,np.matmul(d1,mn))))- np.matmul(d1,mn)/beta))\
                    +0.5*np.trace(np.linalg.matrix_power(np.matmul(A_inv,np.matmul(d1.T,d1)),2))
    g2_alpha_beta = -np.matmul((x - np.matmul(d1,mn)).T,(np.matmul(d1,np.matmul(A_inv,mn)))) +\
                    0.5*np.trace(np.matmul(np.linalg.matrix_power(A_inv,2),np.matmul(d1.T,d1)))

    
    return np.array([g2_alpha,g2_beta,g2_alpha_beta])/num_obs  


In [ ]:



alpha_list = []
beta_list = []
weights_list = []
auc_list = []


bar_e = tqdm(enumerate(epsilon_list), total=len(epsilon_list))
for i_e in bar_e:

    # Fitting the model
    alphas = [None]*n_avg
    betas = [None]*n_avg
    weights = [None]*n_avg
    auc = [None]*n_avg


    bar_avg = tqdm(range(n_avg), total=n_avg, leave=False)
    for i_a in bar_avg:
    


  0%|          | 0/10 [00:00<?, ?it/s]


(136, 136)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [20]:
get_rates_par()

{'dynb':     index type educt1 educt2 product1    ratef   rateb
 0       6    b    GRb    SOS       GS  0.00010  0.0015
 1       3    b     RP    GRb       RG  0.00300  0.0500
 2       4    b     RG    SOS      RGS  0.01000  0.0600
 3       5    b     RP     GS      RGS  0.00450  0.0300
 4       1    b     RP   PLCg      RPL  0.06000  0.2000
 5       2    b     RP  PLCgP     RPLP  0.00600  0.3000
 6       7    b     RP    Shc      RSh  0.09000  0.6000
 7       9    b   RShP    GRb     RShG  0.00300  0.1000
 8      10    b     RP    ShG     RShG  0.00090  0.3000
 9      16    b   RShP     GS    RShGS  0.00900  0.0429
 10     12    b   ShGS     RP    RShGS  0.00024  0.1200
 11     11    b   RShG    SOS    RShGS  0.01000  0.0214
 12      8    b    ShP     RP     RShP  0.00090  0.3000
 13      0    b      R    EGF       Ra  0.00300  0.0600
 14     13    b    ShP    GRb      ShG  0.00300  0.1000
 15     14    b    ShG    SOS     ShGS  0.03000  0.0640
 16     15    b    ShP     GS     ShGS  

In [31]:
[]*10

[]

In [28]:
len(weights_list[])

15